In [1]:
from nltk.parse import DependencyGraph, DependencyEvaluator
from nltk.parse.transitionparser import TransitionParser, Configuration, Transition
import tempfile, os

In [2]:
file = "./UD_Hindi/hi-ud-train.conllu"
fd = open(file)
trainData = []
graph = ""
for line in fd:
    graph = graph + line
    if line.strip() == "":
        trainData.append(graph)
        graph = ""
        
filet = "./UD_Hindi/hi-ud-test.conllu"
fd = open(filet)
testData = []
graph = ""
for line in fd:
    graph = graph + line
    if line.strip() == "":
        testData.append(graph)
        graph = ""

In [3]:
gold_sent = DependencyGraph("""
    Economic  JJ     2      ATT
    news  NN     3       SBJ
    has       VBD       0       ROOT
    little      JJ      5       ATT
    effect   NN     3       OBJ
    on     IN      5       ATT
    financial       JJ       8       ATT
    markets    NNS      6       PC
    .    .      3       PU
    """)
print(gold_sent)

defaultdict(<function DependencyGraph.__init__.<locals>.<lambda> at 0x7fec5fa2e620>,
            {0: {'address': 0,
                 'ctag': 'TOP',
                 'deps': defaultdict(<class 'list'>, {'ROOT': [3]}),
                 'feats': None,
                 'head': None,
                 'lemma': None,
                 'rel': None,
                 'tag': 'TOP',
                 'word': None},
             1: {'address': 1,
                 'ctag': 'JJ',
                 'deps': defaultdict(<class 'list'>, {}),
                 'feats': '',
                 'head': 2,
                 'lemma': 'Economic',
                 'rel': 'ATT',
                 'tag': 'JJ',
                 'word': 'Economic'},
             2: {'address': 2,
                 'ctag': 'NN',
                 'deps': defaultdict(<class 'list'>, {'ATT': [1]}),
                 'feats': '',
                 'head': 3,
                 'lemma': 'news',
                 'rel': 'SBJ',
                 'tag': 'NN

In [4]:
t = DependencyGraph(trainData[0])
print(t)

defaultdict(<function DependencyGraph.__init__.<locals>.<lambda> at 0x7fec135c89d8>,
            {0: {'address': 0,
                 'ctag': 'TOP',
                 'deps': defaultdict(<class 'list'>, {'root': [9], 'ROOT': []}),
                 'feats': None,
                 'head': None,
                 'lemma': None,
                 'rel': None,
                 'tag': 'TOP',
                 'word': None},
             1: {'address': 1,
                 'ctag': 'DET',
                 'deps': defaultdict(<class 'list'>, {}),
                 'feats': 'Case=Nom|Number=Sing|Person=3|PronType=Dem',
                 'head': 2,
                 'lemma': 'yaha',
                 'rel': 'det',
                 'tag': 'DET',
                 'word': 'yaha'},
             2: {'address': 2,
                 'ctag': 'PROPN',
                 'deps': defaultdict(<class 'list'>, {'det': [1], 'case': [3]}),
                 'feats': 'Case=Acc|Gender=Masc|Number=Sing|Person=3',
               

/home/buridi/anaconda3/envs/nlp/lib/python3.6/site-packages/nltk/parse/dependencygraph.py:380: UserWarning: The graph doesn't contain a node that depends on the root element.
  "The graph doesn't contain a node "


In [5]:
trainDataGraph = []
for t in trainData:
    d = DependencyGraph(t)
    trainDataGraph.append(d)
    
testDataGraph = []
for t in testData:
    d = DependencyGraph(t)
    testDataGraph.append(d)    

/home/buridi/anaconda3/envs/nlp/lib/python3.6/site-packages/nltk/parse/dependencygraph.py:380: UserWarning: The graph doesn't contain a node that depends on the root element.
  "The graph doesn't contain a node "


In [6]:
input_file = tempfile.NamedTemporaryFile(prefix='transition_parse.train', dir=tempfile.gettempdir(), delete=False)

In [7]:
parser_std = TransitionParser('arc-standard')

In [8]:
print(', '.join(parser_std._create_training_examples_arc_std(trainDataGraph, input_file)))

 Number of training examples : 500
 Number of valid (projective) examples : 476
SHIFT, LEFTARC:det, SHIFT, RIGHTARC:case, SHIFT, SHIFT, LEFTARC:advmod, SHIFT, LEFTARC:amod, LEFTARC:nmod, SHIFT, RIGHTARC:case, SHIFT, RIGHTARC:case, SHIFT, LEFTARC:nmod, SHIFT, RIGHTARC:cop, SHIFT, RIGHTARC:punct, RIGHTARC:root, SHIFT, SHIFT, SHIFT, LEFTARC:compound, SHIFT, RIGHTARC:case, SHIFT, LEFTARC:nsubj, LEFTARC:obj, SHIFT, RIGHTARC:aux, SHIFT, RIGHTARC:punct, RIGHTARC:root, SHIFT, SHIFT, SHIFT, LEFTARC:compound, LEFTARC:nmod, SHIFT, SHIFT, LEFTARC:nummod, LEFTARC:nsubj, SHIFT, RIGHTARC:cop, SHIFT, RIGHTARC:punct, RIGHTARC:root, SHIFT, SHIFT, SHIFT, LEFTARC:nummod, LEFTARC:nmod, SHIFT, RIGHTARC:cop, SHIFT, SHIFT, SHIFT, SHIFT, LEFTARC:compound, LEFTARC:amod, SHIFT, RIGHTARC:case, SHIFT, SHIFT, RIGHTARC:mark, SHIFT, RIGHTARC:mark, SHIFT, SHIFT, SHIFT, LEFTARC:compound, LEFTARC:nummod, SHIFT, LEFTARC:nsubj, LEFTARC:advcl, LEFTARC:obl, LEFTARC:cc, RIGHTARC:conj, SHIFT, RIGHTARC:punct, RIGHTARC:root, SH

In [9]:
print(parser_std._create_training_examples_arc_std(trainDataGraph, input_file))

 Number of training examples : 500
 Number of valid (projective) examples : 476
['SHIFT', 'LEFTARC:det', 'SHIFT', 'RIGHTARC:case', 'SHIFT', 'SHIFT', 'LEFTARC:advmod', 'SHIFT', 'LEFTARC:amod', 'LEFTARC:nmod', 'SHIFT', 'RIGHTARC:case', 'SHIFT', 'RIGHTARC:case', 'SHIFT', 'LEFTARC:nmod', 'SHIFT', 'RIGHTARC:cop', 'SHIFT', 'RIGHTARC:punct', 'RIGHTARC:root', 'SHIFT', 'SHIFT', 'SHIFT', 'LEFTARC:compound', 'SHIFT', 'RIGHTARC:case', 'SHIFT', 'LEFTARC:nsubj', 'LEFTARC:obj', 'SHIFT', 'RIGHTARC:aux', 'SHIFT', 'RIGHTARC:punct', 'RIGHTARC:root', 'SHIFT', 'SHIFT', 'SHIFT', 'LEFTARC:compound', 'LEFTARC:nmod', 'SHIFT', 'SHIFT', 'LEFTARC:nummod', 'LEFTARC:nsubj', 'SHIFT', 'RIGHTARC:cop', 'SHIFT', 'RIGHTARC:punct', 'RIGHTARC:root', 'SHIFT', 'SHIFT', 'SHIFT', 'LEFTARC:nummod', 'LEFTARC:nmod', 'SHIFT', 'RIGHTARC:cop', 'SHIFT', 'SHIFT', 'SHIFT', 'SHIFT', 'LEFTARC:compound', 'LEFTARC:amod', 'SHIFT', 'RIGHTARC:case', 'SHIFT', 'SHIFT', 'RIGHTARC:mark', 'SHIFT', 'RIGHTARC:mark', 'SHIFT', 'SHIFT', 'SHIFT', 'LEFTA

In [10]:
parser_std.train(trainDataGraph,'temp.arcstd.model', verbose=False)

 Number of training examples : 500
 Number of valid (projective) examples : 476


In [11]:
os.remove(input_file.name)

In [12]:
result = parser_std.parse(testDataGraph, 'temp.arcstd.model')

In [13]:
de = DependencyEvaluator(result, testDataGraph)

In [14]:
print(de)